In [1]:
import os
import sys
sys.path.append(os.getcwd() + "/src")

# Seeing if there is Threshold for CPC codes

In [2]:
from scoring.score_dataset import run_decoder
import json
from global_params import params
from CPC import generate_random
import numpy as np
import utils

error_rates = [0.005, 0.01, 0.015, 0.02]
R = 0.2
n_bits = [30, 60, 120, 240]
n_samples = 100

def get_key(n, m, err):
	return f"({n},{m}):{err}"

best_dict = {}
best_pc = {}
for n_bit in n_bits:
	for e in error_rates:
		m = int((1 - R) * n_bit)
		k = get_key(n_bit, m, e)
		for _ in range(n_samples):
			pc, _, _, _ = generate_random.random_cpc(n_bits=n_bit, n_checks=m)
			wsr = run_decoder(pc, np.ones(pc.shape[-1]) * e)
			if k not in best_dict:
				best_dict[k] = 1
			if 1 - wsr < best_dict[k]:
				print(f"For n={n_bit}, m={m}, e={e}, we have error rate {1 - wsr}")
				best_dict[k] = 1 - wsr
				best_pc[k] = pc

out_json = {}
out_json['err_rates'] = best_dict
out_json['corresponding_pc'] = best_pc

outfile = 'tmp_random_thresh.json'
with open(outfile, "w") as outfile:
    print("WRITING", out_json)
    json.dump(out_json, outfile, cls=utils.NpEncoder)
